In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFMinerLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryStore

import os

In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain import PromptTemplate

In [3]:
# !pip install --upgrade --quiet  langchain sentence_transformers

In [4]:
# !pip install chromadb
#!pip install pypdf
#!pip install pdfminer.six
# !pip install huggingface_hub

In [3]:
WORKING_DIR = os.getcwd()

FILE_NAME = "test.pdf"
FILE_PATH = os.path.join(WORKING_DIR, "Data", FILE_NAME)
FILE_PATH

'C:\\Users\\alber\\Documents\\Projects\\BookRecommendation\\Data\\test.pdf'

In [4]:
loader = PDFMinerLoader(FILE_PATH)
#loader = PyPDFLoader(FILE_PATH)
pages = loader.load()

In [5]:
len(pages[0].page_content)

2

In [6]:
len(pages)

1

In [21]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/11/pg11-images.html")

pages = loader.load()

In [22]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=735, separators=["\n\n", "\n", ". ", "! ", "? "])
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
#child_splitter = RecursiveCharacterTextSplitter(chunk_size=100)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=10
    )

In [51]:
vectorstore = "None"
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=HuggingFaceEmbeddings()
)

store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=text_splitter,
)

In [23]:
docs = parent_splitter.split_documents(pages)

In [24]:
db = Chroma.from_documents(docs, HuggingFaceEmbeddings())

In [12]:
retriever.add_documents(pages, ids=None)

In [13]:
len(list(store.yield_keys()))

526

In [25]:
retrieved_docs = db.similarity_search("What didn't Alice think so very much about?", k=10)

In [26]:
print(len(retrieved_docs))

10


In [27]:
retrieved_docs

[Document(page_content='But her sister sat still just as she left her, leaning her head on her hand,\r\nwatching the setting sun, and thinking of little Alice and all her wonderful\r\nAdventures, till she too began dreaming after a fashion, and this was her\r\ndream:—\r\n\n\r\nFirst, she dreamed of little Alice herself, and once again the tiny hands were\r\nclasped upon her knee, and the bright eager eyes were looking up into\r\nhers—she could hear the very tones of her voice, and see that queer\r\nlittle toss of her head to keep back the wandering hair that would\r\nalways get into her eyes—and still as she listened, or seemed to listen,\r\nthe whole place around her became alive with the strange creatures of her\r\nlittle sister’s dream.', metadata={'language': 'en', 'source': 'https://www.gutenberg.org/cache/epub/11/pg11-images.html', 'title': 'The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll'}),
 Document(page_content="Lastly, she pictured to hersel

In [76]:
new_retriever = db.as_retriever()

In [26]:
sub_docs = db.similarity_search("What was Alice thinking about a daisy-chain?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [34]:
#print(sub_docs[0].page_content)

In [80]:
new_retriever.get_relevant_documents("What was Alice thinking about a daisy-chain?")

[Document(page_content='So she was considering in her own mind (as well as she could, for the hot day\r\nmade her feel very sleepy and stupid), whether the pleasure of making a\r\ndaisy-chain would be worth the trouble of getting up and picking the daisies,\r\nwhen suddenly a White Rabbit with pink eyes ran close by her.', metadata={'language': 'en', 'source': 'https://www.gutenberg.org/cache/epub/11/pg11-images.html', 'title': 'The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll'}),
 Document(page_content='So she was considering in her own mind (as well as she could, for the hot day\r\nmade her feel very sleepy and stupid), whether the pleasure of making a\r\ndaisy-chain would be worth the trouble of getting up and picking the daisies,\r\nwhen suddenly a White Rabbit with pink eyes ran close by her.', metadata={'language': 'en', 'source': 'https://www.gutenberg.org/cache/epub/11/pg11-images.html', 'title': 'The Project Gutenberg eBook of Alice’s Adventure

In [15]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

 ········


In [16]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [17]:
#repo_id = "google/flan-t5-large" 
repo_id = "declare-lab/flan-alpaca-large"

In [18]:
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 1000}
)

C:\Users\alber\anaconda3\envs\LLMApp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [29]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based on the provided context.

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [31]:
resp = document_chain.invoke({
    "input": "What didn't Alice think so very much about?",
    "context": retrieved_docs
})

In [33]:
type(resp)

str

In [77]:
retrieval_chain = create_retrieval_chain(new_retriever, document_chain)

In [9]:
response = retrieval_chain.invoke({"input": "What didn't Alice think so very much about?"})
print(response["answer"])

NameError: name 'retrieval_chain' is not defined

In [8]:
len(response["answer"])

NameError: name 'response' is not defined

In [67]:
response

{'input': 'Explain what is a vector embedding?',
 'context': [Document(page_content='(6.4) ...spinach sauteed with garlic over rice...\n(6.5) ...chard stems and leaves are delicious...\n(6.6) ...collard greens and other salty leafy greens\n\nThe fact that ongchoi occurs with words like rice and garlic and delicious and\nsalty, as do words like spinach, chard, and collard greens might suggest that ongchoi\nis a leafy green similar to these other leafy greens.1 We can do the same thing\ncomputationally by just counting words in the context of ongchoi.\n\nThe idea of vector semantics is to represent a word as a point in a multidimen-\nsional semantic space that is derived (in ways we’ll see) from the distributions of\nword neighbors. Vectors for representing words are called embeddings (although\nthe term is sometimes more strictly applied only to dense vectors like word2vec\n(Section 6.8), rather than sparse tf-idf or PPMI vectors (Section 6.3-Section 6.6)).\nThe word “embedding” derives

In [68]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
            If you don't know the answer, just say that you don't know, don't try to make up an answer.
            If the question is similar to [Talk me about the document], 
            the response should be a summary commenting on the most important points about the document


            {context}
            Question: {question}
            """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [78]:
qa_chain = load_qa_chain(llm, chain_type="stuff", prompt = PROMPT)

In [79]:
question = "What is the a word embedding"

In [80]:
response = qa_chain({"input_documents": retrieved_docs, "question": question}, return_only_outputs=True)

In [81]:
response

{'output_text': 'In LSA, the word embedding seems to have described the mapping from the space of'}

In [163]:
retrieved_docs

[Document(page_content='These word representations are also the ﬁrst example in this book of repre-\nsentation learning, automatically learning useful representations of the input text.\nFinding such self-supervised ways to learn representations of the input, instead of\ncreating representations by hand via feature engineering, is an important focus of\nNLP research (Bengio et al., 2013).\n\ndistributional\nhypothesis', metadata={'source': 'C:\\Users\\alber\\Documents\\Projects\\BookRecommendation\\Data\\embeddings.pdf'})]

In [10]:
context = """Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?”
So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.
In another moment down went Alice after it, never once considering how in the world she was to get out again.
The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.
Either the well was very deep, or she fell very slowly, for she had plenty of time as she went down to look about her and to wonder what was going to happen next. First, she tried to look down and make out what she was coming to, but it was too dark to see anything; then she looked at the sides of the well, and noticed that they were filled with cupboards and book-shelves; here and there she saw maps and pictures hung upon pegs. She took down a jar from one of the shelves as she passed; it was labelled “ORANGE MARMALADE”, but to her great disappointment it was empty: she did not like to drop the jar for fear of killing somebody underneath, so managed to put it into one of the cupboards as she fell past it."""

In [12]:
from langchain.docstore.document import Document

In [13]:
doc = Document(page_content=context)

In [14]:
doc

Document(page_content='Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?”\nSo she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.\nThere was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket,

In [16]:
resp = document_chain.invoke({
    "input": "How did Alice end up in Wonderland?",
    "context": [doc]
})

resp

'Alice ended up in Wonderland by running after a rabbit and falling down a deep'

In [18]:
len(resp)

78

In [20]:
len(context)

2372

In [39]:
resp = document_chain.invoke({
    "input": "Please provide a summary of context",
    "context": [doc]
})

resp

'Alice was bored and decided to make a daisy-chain. She was surprised when'

In [42]:
import requests

sess = requests.session()

In [43]:
query = "How was the jar on the shelves labelled?"
web_path = "https://www.gutenberg.org/cache/epub/11/pg11-images.html"

json_file = {"web_path": web_path}#, "query": query}
req = sess.post("http://127.0.0.1:5000/predict", json=json_file)

In [44]:
req

<Response [200]>

In [45]:
req.json()

{'response': 'Initialized',
 'test': ["The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Project Gutenberg eBook of Alice's Adventures in Wonderland\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\nTitle: Alice's Adventures in Wonderland\n\nAuthor: Lewis Carroll",
  "Author: Lewis Carroll\n\nRelease date: June 27, 2008 [eBook #11]\r\n                Most recently updated: February 4, 2024\nLanguage: English\nCredits: Arthur DiBianca and David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK ALICE'S

In [37]:
query = "What didn't Alice think so very much about?"
web_path = "https://www.gutenberg.org/cache/epub/11/pg11-images.html"

json_file = {"web_path": web_path, "query": query}

In [40]:
req = sess.post("http://127.0.0.1:5000/predict", json=json_file)

In [41]:
req.json()

{'response': "Alice didn't think so much about her sister.", 'test': None}

In [17]:
sess.__dict__

{'headers': {'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'},
 'auth': None,
 'proxies': {},
 'hooks': {'response': []},
 'params': {},
 'stream': False,
 'verify': True,
 'cert': None,
 'max_redirects': 30,
 'trust_env': True,
 'cookies': <RequestsCookieJar[Cookie(version=0, name='session', value='eyJfcGVybWFuZW50Ijp0cnVlLCJtb2RlbCI6MX0.ZcUlIA.iagyYXt_kIR2zC4zGzF3Dhc1ygU', port=None, port_specified=False, domain='127.0.0.1', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1707419512, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>,
 'adapters': OrderedDict([('https://',
               <requests.adapters.HTTPAdapter at 0x157ffc12450>),
              ('http://', <requests.adapters.HTTPAdapter at 0x157810f76d0>)])}

In [21]:
req.cookies

<RequestsCookieJar[Cookie(version=0, name='session', value='eyJfcGVybWFuZW50Ijp0cnVlLCJtb2RlbCI6MX0.ZcUlIA.iagyYXt_kIR2zC4zGzF3Dhc1ygU', port=None, port_specified=False, domain='127.0.0.1', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1707419512, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [23]:
sess.__dict__

{'headers': {'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'},
 'auth': None,
 'proxies': {},
 'hooks': {'response': []},
 'params': {},
 'stream': False,
 'verify': True,
 'cert': None,
 'max_redirects': 30,
 'trust_env': True,
 'cookies': <RequestsCookieJar[Cookie(version=0, name='session', value='eyJfcGVybWFuZW50Ijp0cnVlLCJtb2RlbCI6MX0.ZcUlIA.iagyYXt_kIR2zC4zGzF3Dhc1ygU', port=None, port_specified=False, domain='127.0.0.1', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1707419512, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>,
 'adapters': OrderedDict([('https://',
               <requests.adapters.HTTPAdapter at 0x157ffc12450>),
              ('http://', <requests.adapters.HTTPAdapter at 0x157810f76d0>)])}